--------------
Get Data:

In [256]:
import copy

In [257]:
g = lambda x: [y.strip() for y in x]

In [258]:
classRooms = [
    "A1.04 	41",
    "A1.06 	22",
    "A1.08 	20",
    "A1.10 	56",
    "B0.201 	48",
    "C0.110 	117",
    "C1.112 	60",]

classRooms = [g(x.split("\t")) for x in classRooms]

In [259]:
subjects = """Advanced Heuristics 	1 	0 	nvt 	1 	10
Algoritmen en complexiteit 	1 	1 	25 	1 	25
Analysemethoden en -technieken 	1 	0 	nvt 	0 	nvt
Architectuur en computerorganisatie 	2 	0 	nvt 	0 	nvt
Autonomous Agents 2 	2 	1 	10 	1 	10
Bioinformatica 	3 	1 	20 	1 	20
Calculus 2 	1 	1 	40 	0 	nvt
Collectieve Intelligentie 	3 	1 	20 	1 	20
Compilerbouw 	2 	1 	40 	1 	40
Compilerbouw (practicum) 	0 	0 	nvt 	1 	15
Data Mining 	2 	1 	10 	1 	10
Databases 2 	1 	1 	40 	0 	nvt
Heuristieken 1 	1 	1 	25 	0 	nvt
Heuristieken 2 	1 	1 	20 	0 	nvt
Informatie- en organisatieontwerp 	2 	1 	15 	1 	15
Interactie-ontwerp 	2 	0 	nvt 	0 	nvt
Kansrekenen 2 	2 	0 	nvt 	0 	nvt
Lineaire Algebra 	2 	0 	nvt 	0 	nvt
Machine Learning 	2 	0 	nvt 	0 	nvt
Moderne Databases 	1 	1 	20 	1 	20
Netwerken en systeembeveiliging 	0 	0 	nvt 	1 	20
Programmeren in Java 2 	0 	0 	nvt 	1 	20
Project Genetic Algorithms 	0 	0 	nvt 	1 	15
Project Numerical Recipes 	0 	0 	nvt 	1 	15
Reflectie op de digitale cultuur 	2 	1 	20 	0 	nvt
Software engineering 	1 	1 	40 	1 	40
Technology for games 	2 	1 	20 	0 	nvt
Webprogrammeren en databases 	2 	1 	20 	1 	20
Zoeken, sturen en bewegen 	0 	0 	nvt 	1 	15"""

subjects = [g(x.split("\t")) for x in subjects.split("\n")]

In [260]:
students = []

import csv

with open('studenten_roostering.csv', 'r') as csvfile:
    r = csv.reader(csvfile, delimiter=',', quotechar='|')
    
    # Skip title
    next(r, None)
    
    for row in r:
        students.append(row)

--------------
Classes:

In [261]:
class ClassRoom:
    def __init__(self, room_number, capacity):
        self.room_number = room_number
        self.capacity = capacity
    
    def __str__(self):
        return self.room_number

In [262]:
class Subject:
    def __init__(self, name, n_lectures, n_workLectures, w_maxStud, 
                 n_praticas, p_maxStud):
        
        self.name = name
        
        self.lectures = []
        self.workLectures = []
        self.praticas = []
        
        for i in range(int(n_lectures)):
            self.lectures.append(Lecture("Lecture", i, "nvt"))
            
        for i in range(int(n_workLectures)):
            self.workLectures.append(Lecture("WorkLecture", i, w_maxStud))
            
        for i in range(int(n_praticas)):
            self.praticas.append(Lecture("Pratica", i, p_maxStud))
        
        self.students = []
        
    def __str__(self):
        return self.name
    
    def assignStudentsToLectures(self):
        if len(self.workLectures) and self.workLectures[0].maxStud:
            # Calculate number of groups and the number of students per lecture then round down
            nGroups = int(len(self.students) / self.workLectures[0].maxStud)
            wLectureSize = int(len(self.students) / nGroups)
            
            if nGroups > 1:
                newWorkLectures = []
                
                # For every work lecture, there are the same groups that need to be filled
                # But these groups also need to be created
                for wLecture in self.workLectures:
                    for groupNumber in range(nGroups):
                        nLecture = copy.copy(wLecture)
                        nLecture.group = groupNumber
                        
                        nLecture.students = self.students[groupNumber*wLectureSize:(groupNumber+1)*wLectureSize]
                                                
                        newWorkLectures.append(nLecture)
                        
                        
                self.workLectures = newWorkLectures
            
        if len(self.praticas) and self.praticas[0].maxStud:
            # Calculate number of groups and the number of students per lecture then round down
            nGroups = int(len(self.students) / self.praticas[0].maxStud)
            pLectureSize = int(len(self.students) / nGroups)
            
            if nGroups > 1:
                newPraticaLectures = []
                
                # For every work lecture, there are the same groups that need to be filled
                # But these groups also need to be created
                for pLecture in self.praticas:
                    for groupNumber in range(nGroups):
                        nLecture = copy.copy(pLecture)
                        nLecture.group = groupNumber
                        
                        nLecture.students = self.students[groupNumber*pLectureSize:(groupNumber+1)*pLectureSize]
                                                
                        newPraticaLectures.append(nLecture)
                        
                        
                self.praticas = newPraticaLectures

In [263]:
class Lecture:
    def __init__(self, name, lecture_number, maxStud):
        self.lecture_number = lecture_number
        self.name = name
        
        if maxStud == "nvt":
            self.maxStud = 0
        else:
            self.maxStud = int(maxStud)
        
        self.students = []
        self.group = 0
        
    def __str__(self):
        return "Name: %s Lecture number: %s Group: %s maxStud: %s" % (self.name, self.lecture_number, self.group, self.maxStud)
    

In [264]:
class Student:
    def __init__(self, surname, name, studentId, subject1, subject2, 
                 subject3, subject4, subject5, subject_dct):
        
        self.surname = surname
        self.name = name
        self.studentId = studentId
        
        self.subjectNames = []
        self.__addSubject(subject1)
        self.__addSubject(subject2)
        self.__addSubject(subject3)
        self.__addSubject(subject4)
        self.__addSubject(subject5)
        
        self.subjects = []
        
        self.__cleanUp()
        self.__fillInSubject(subject_dct)
        self.__addStudentToSubject()      
        
    def __str__(self):
        return "%s %s %s" % (self.name, self.surname, self.studentId)
    
    def __addSubject(self, subject):
        if subject != "":
            self.subjectNames.append(subject)
    
    def __addStudentToSubject(self):
        for subject in self.subjects:
            subject.students.append(self)
    
    def __cleanUp(self):
        # Fixes CSV errors
        
        for i, x in enumerate(self.subjectNames):
            if x == "Zoeken":
                self.subjectNames[i] = "Zoeken, sturen en bewegen"
            if x == " sturen en bewegen":
                del self.subjectNames[i]
            if x == "Compilerbouw practicum":
                self.subjectNames[i] = "Compilerbouw (practicum)"
            if x == "Informatie- en organsatieontwerp":
                self.subjectNames[i] = "Informatie- en organisatieontwerp"
                
    def __fillInSubject(self, subject_dct):
        for subjectName in self.subjectNames:
            self.subjects.append(subject_dct[subjectName])

--------------
Transform data:

In [265]:
classRooms = [ClassRoom(x[0], x[1]) for x in classRooms]

subjects = [Subject(x[0], x[1], x[2], x[3], x[4], x[5]) for x in subjects]
subject_dct = {x.__str__(): x for x in subjects}

students = [Student(x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], subject_dct) 
            for x in students]


In [266]:
# Assign students to lectures
for x in subjects:
    x.assignStudentsToLectures()

In [279]:
i=4

print(subjects[i])
print("\n".join([x.__str__() for x in subjects[i].praticas]))
[(x.group, [y.__str__() for y in x.students]) for x in subjects[i].praticas]

Autonomous Agents 2
Name: Pratica Lecture number: 0 Group: 0 maxStud: 10
Name: Pratica Lecture number: 0 Group: 1 maxStud: 10


[(0,
  ['Abdallah Beulen 22871048',
   'Tarah Broerse 62872965',
   'Virginie Brunsveld 5517869',
   'Herwin Ester 74131924',
   'Jurrian Fekkes 27298319',
   'Deacon Gielissen 11915563',
   'Umar Grift 70113753',
   'Bo Hilhorst 36508571',
   'Esmée Hooijmaijers 62356458',
   'Danique Knibbe 16097870',
   'Resul Kuijpers 54026378']),
 (1,
  ['Roan Meulmeester 68409833',
   'André Putters 37110913',
   'Luciana Putters 61611295',
   'Angelica Sarikaya 52552718',
   'Emrah Schrama 44901052',
   'Shaya Snelder 19858730',
   'Deejay van der Geer 51566595',
   'Tamimount van Herten 88627897',
   'Lammigje van Loveren 39459642',
   'Rhona Vromans 34806836',
   'Filipe Wiersema 72796858'])]

In [278]:
i=7

print(subjects[i])
print("\n".join([x.__str__() for x in subjects[i].workLectures]))
[(x.group, [y.__str__() for y in x.students]) for x in subjects[i].workLectures]

Collectieve Intelligentie
Name: WorkLecture Lecture number: 0 Group: 0 maxStud: 20
Name: WorkLecture Lecture number: 0 Group: 1 maxStud: 20
Name: WorkLecture Lecture number: 0 Group: 2 maxStud: 20


[(0,
  ['Evi Appel 13534377',
   'Rhona Bakkum 50184318',
   'Wafa Bart 64296965',
   'Sharan Baselmans 42778953',
   'Edin Bastings 82689818',
   'Ravi Bauer 51741554',
   'Marrie Bekker 97135019',
   'Ella Bischoff 46052388',
   'Marco Bloemert 1075391',
   'Femia Brink 45556081',
   'Virginie Brunsveld 5517869',
   'Laurentia Cooijmans 33760909',
   'Marije de Vreeze 64393888',
   'Brigitta Diallo 50143947',
   'Sohrab Geers 87758823',
   'Wichard Hospers 31416266',
   'Jenske Jongejan 15902055',
   'Kimmy Kesteloo 34773179',
   'Ylse Klarenbeek 62701040',
   'Matz Koning 48601268']),
 (1,
  ['Titia Korbee 86291592',
   'Vincent Kros 22699561',
   'Buck Lantinga 23547619',
   'Björn Leijen 3233378',
   'Hetty Lindhout 1696500',
   'Valéry Majoor 58522654',
   'Miquel Metzelaar 86000192',
   'Marcellus Mijnheer 4783858',
   'Jorren Modderkolk 72577292',
   'Shannon Mullenders 54310617',
   'Inger Notermans 13092455',
   'Casimir Overbeeke 17575088',
   'Solane Paulissen 1665980',
   

In [269]:
# Laat alle studenten in Algoritmen en complexiteit zien:
i = 7

print(subjects[i])
[x.__str__() for x in subjects[i].students]

Collectieve Intelligentie


['Evi Appel 13534377',
 'Rhona Bakkum 50184318',
 'Wafa Bart 64296965',
 'Sharan Baselmans 42778953',
 'Edin Bastings 82689818',
 'Ravi Bauer 51741554',
 'Marrie Bekker 97135019',
 'Ella Bischoff 46052388',
 'Marco Bloemert 1075391',
 'Femia Brink 45556081',
 'Virginie Brunsveld 5517869',
 'Laurentia Cooijmans 33760909',
 'Marije de Vreeze 64393888',
 'Brigitta Diallo 50143947',
 'Sohrab Geers 87758823',
 'Wichard Hospers 31416266',
 'Jenske Jongejan 15902055',
 'Kimmy Kesteloo 34773179',
 'Ylse Klarenbeek 62701040',
 'Matz Koning 48601268',
 'Titia Korbee 86291592',
 'Vincent Kros 22699561',
 'Buck Lantinga 23547619',
 'Björn Leijen 3233378',
 'Hetty Lindhout 1696500',
 'Valéry Majoor 58522654',
 'Miquel Metzelaar 86000192',
 'Marcellus Mijnheer 4783858',
 'Jorren Modderkolk 72577292',
 'Shannon Mullenders 54310617',
 'Inger Notermans 13092455',
 'Casimir Overbeeke 17575088',
 'Solane Paulissen 1665980',
 'Xander Peijs 96104167',
 'Matthijs Pleiter 77240949',
 'Romello Reintjes 294747